In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import scipy.signal as signal
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import os
import pandas as pd
from scipy.stats import skew, kurtosis
import seaborn as sns
import gc

# STEP 1: Data Loading and Preprocessing
class EEGDataset(Dataset):
    def __init__(self, data_dict, label_mapping_file=None, transform=None):
        self.dataset = data_dict['dataset']
        self.labels_list = data_dict['labels']  # List of label IDs
        self.transform = transform

        # Load the mapping from label ID to text if provided
        self.label_id_to_text = {}
        if label_mapping_file and os.path.exists(label_mapping_file):
            with open(label_mapping_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        label_id = parts[0]
                        label_text = ' '.join(parts[1:])
                        self.label_id_to_text[label_id] = label_text

        # Create mapping from label ID to index
        unique = sorted(set(self.labels_list))
        self.label_to_idx = {lbl:i for i,lbl in enumerate(unique)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        eeg_data = sample['eeg_data']  # Shape: [channels, time_points]
        label_id = sample['label']  # This is a string ID like 'n02510455'

        # Get label text if available
        label_text = self.label_id_to_text.get(label_id, label_id)

        # Convert string label to numerical index
        label_idx = self.label_to_idx.get(label_id, 0)

        if self.transform:
            eeg_data = self.transform(eeg_data)

        return eeg_data, label_idx, label_text

# EEG Signal Preprocessing
class EEGPreprocessor:
    def __init__(self, sampling_rate=1000, notch_freq=50, bandpass_low=0.5, bandpass_high=70):
        self.sampling_rate = sampling_rate
        self.notch_freq = notch_freq
        self.bandpass_low = bandpass_low
        self.bandpass_high = bandpass_high

        # Pre-compute filter coefficients to avoid recomputation
        self.sos = signal.butter(
            N=4,
            Wn=[self.bandpass_low, self.bandpass_high],
            btype='bandpass',
            fs=self.sampling_rate,
            output='sos'
        )
        self.b_notch, self.a_notch = signal.iirnotch(self.notch_freq, 30, self.sampling_rate)

    def __call__(self, eeg_data):
        # Convert to numpy if it's a tensor
        if isinstance(eeg_data, torch.Tensor):
            eeg_data = eeg_data.numpy()

        # Transpose to [time, channels] for easier processing
        eeg_data = eeg_data.T

        # Apply bandpass filter
        eeg_filtered = self._bandpass_filter(eeg_data)

        # Apply notch filter (to remove power line interference)
        eeg_filtered = self._notch_filter(eeg_filtered)

        # Re-reference to common average
        eeg_filtered = self._common_average_reference(eeg_filtered)

        # Z-score normalization
        eeg_normalized = self._normalize(eeg_filtered)

        # Transpose back to [channels, time]
        return torch.tensor(eeg_normalized.T, dtype=torch.float32)

    def _bandpass_filter(self, data):
        # Apply forward-backward filtering for zero phase distortion
        return signal.sosfiltfilt(self.sos, data, axis=0)

    def _notch_filter(self, data):
        return signal.filtfilt(self.b_notch, self.a_notch, data, axis=0)

    def _common_average_reference(self, data):
        # Subtract the mean across all channels at each time point
        return data - np.mean(data, axis=1, keepdims=True)

    def _normalize(self, data):
        # Z-score normalization for each channel
        return (data - np.mean(data, axis=0)) / (np.std(data, axis=0) + 1e-10)

# STEP 2: Feature Extraction
class FeatureExtractor:
    def __init__(self, sampling_rate=1000):
        self.sampling_rate = sampling_rate

        # Define frequency bands
        self.freq_bands = {
          'delta': (1, 4),       # Adjusted lower bound to reduce DC components
          'theta': (4, 8),       # Standard theta for cognitive processing
          'alpha_low': (8, 10),  # Lower alpha - attention/inhibition
          'alpha_high': (10, 13),# Higher alpha - semantic processing
          'beta_low': (13, 20),  # Lower beta - motor preparation
          'beta_high': (20, 30), # Higher beta - active processing/cognition
          'gamma_low': (30, 60), # Expanded gamma_low for visual processing
          'gamma_mid': (60, 90), # Added gamma_mid for binding
          'gamma_high': (90, 120)# Higher gamma for fine perceptual binding
      }

    def extract_features(self, eeg_data):
        """
        Extract time and frequency domain features from EEG data

        Args:
            eeg_data: EEG data of shape [channels, time_points]

        Returns:
            features: Dictionary of extracted features
        """
        features = {}

        # Time domain features
        features.update(self._extract_time_domain_features(eeg_data))

        # Frequency domain features
        features.update(self._extract_frequency_domain_features(eeg_data))

        # Connectivity features - can improve classification accuracy
        features.update(self._extract_connectivity_features(eeg_data))

        # Convert dictionary to vector
        feature_vector = []
        for key, value in features.items():
            if isinstance(value, np.ndarray):
                feature_vector.append(value.flatten())
            else:
                feature_vector.append(np.array([value]).flatten())

        return np.concatenate(feature_vector)

    def _extract_time_domain_features(self, eeg_data):
        features = {}

        # Statistical features
        features['mean'] = np.mean(eeg_data, axis=1)
        features['var'] = np.var(eeg_data, axis=1)
        features['skewness'] = skew(eeg_data, axis=1)
        features['kurtosis'] = kurtosis(eeg_data, axis=1)
        features['max'] = np.max(eeg_data, axis=1)
        features['min'] = np.min(eeg_data, axis=1)
        features['peak_to_peak'] = features['max'] - features['min']  # Reuse computed values
        features['rms'] = np.sqrt(np.mean(np.square(eeg_data), axis=1))
        features['zero_crossings'] = np.sum(np.diff(np.signbit(eeg_data), axis=1), axis=1)

        # Hjorth parameters
        features.update(self._compute_hjorth_parameters(eeg_data))

        return features

    def _compute_hjorth_parameters(self, eeg_data):
        """Compute Hjorth parameters: Activity, Mobility, and Complexity"""
        features = {}

        # First derivative
        diff1 = np.diff(eeg_data, axis=1)
        # Second derivative
        diff2 = np.diff(diff1, axis=1)

        # Activity: variance of the signal
        features['activity'] = np.var(eeg_data, axis=1)

        # Mobility: sqrt(variance of first derivative / variance of signal)
        var_diff1 = np.var(diff1, axis=1)
        mobility1 = np.sqrt(var_diff1 / (features['activity'] + 1e-10))
        features['mobility'] = mobility1

        # Complexity: mobility of first derivative / mobility of signal
        var_diff2 = np.var(diff2, axis=1)
        mobility2 = np.sqrt(var_diff2 / (var_diff1 + 1e-10))
        features['complexity'] = mobility2 / (mobility1 + 1e-10)

        return features

    def _extract_frequency_domain_features(self, eeg_data):
        features = {}

        # Compute power spectral density with Welch's method
        nperseg = min(256, eeg_data.shape[1] // 4)  # Adaptive window size
        freqs, psd = signal.welch(eeg_data, fs=self.sampling_rate,
                                 nperseg=nperseg,
                                 noverlap=nperseg // 2,
                                 axis=1)

        # Calculate total power once
        total_power = np.sum(psd, axis=1) + 1e-10

        # Band powers and their ratios
        for band_name, (low_freq, high_freq) in self.freq_bands.items():
            # Find frequencies in the band
            idx_band = np.logical_and(freqs >= low_freq, freqs <= high_freq)
            # Calculate band power
            band_power = np.sum(psd[:, idx_band], axis=1)
            features[f'{band_name}_power'] = band_power

            # Calculate relative band power
            features[f'{band_name}_rel_power'] = band_power / total_power

        # Spectral edge frequency (95%)
        features['sef_95'] = self._compute_spectral_edge_frequency(freqs, psd, 0.95)

        # Spectral entropy
        features['spectral_entropy'] = self._compute_spectral_entropy(psd)

        # Spectral peak frequency and power
        peak_freqs = freqs[np.argmax(psd, axis=1)]
        peak_powers = np.max(psd, axis=1)
        features['peak_freq'] = peak_freqs
        features['peak_power'] = peak_powers

        return features

    def _compute_spectral_edge_frequency(self, freqs, psd, edge=0.95):
        """Compute frequency below which edge% of power resides"""
        sef = np.zeros(psd.shape[0])
        for i in range(psd.shape[0]):
            # Cumulative sum of PSD
            cumsum = np.cumsum(psd[i]) / (np.sum(psd[i]) + 1e-10)
            # Find frequency below which edge% of power resides
            idx = np.where(cumsum >= edge)[0]
            if len(idx) > 0:
                sef[i] = freqs[idx[0]]
            else:
                sef[i] = freqs[-1]
        return sef

    def _compute_spectral_entropy(self, psd):
        """Compute spectral entropy"""
        entropy = np.zeros(psd.shape[0])
        for i in range(psd.shape[0]):
            # Normalize PSD
            psd_norm = psd[i] / (np.sum(psd[i]) + 1e-10)
            # Calculate entropy
            entropy[i] = -np.sum(psd_norm * np.log2(psd_norm + 1e-10))
        return entropy

    def _extract_connectivity_features(self, eeg_data):
        """Extract connectivity features between EEG channels"""
        features = {}

        # Number of channels
        n_channels = eeg_data.shape[0]

        # Calculate correlation matrix
        corr_matrix = np.corrcoef(eeg_data)

        # Extract upper triangle (excluding diagonal)
        upper_tri_idx = np.triu_indices(n_channels, k=1)
        correlations = corr_matrix[upper_tri_idx]

        # Basic statistics of correlations
        features['mean_corr'] = np.mean(correlations)
        features['std_corr'] = np.std(correlations)
        features['max_corr'] = np.max(correlations)
        features['min_corr'] = np.min(correlations)

        # Phase synchronization - simplified version using Hilbert transform
        analytic_signal = signal.hilbert(eeg_data, axis=1)
        instantaneous_phase = np.angle(analytic_signal)

        # Calculate phase differences between adjacent channels
        phase_diff = np.zeros((n_channels-1,) + instantaneous_phase.shape[1:])
        for i in range(n_channels-1):
            phase_diff[i] = instantaneous_phase[i+1] - instantaneous_phase[i]

        # Phase locking value (PLV)
        plv_values = np.abs(np.mean(np.exp(1j * phase_diff), axis=1))
        features['mean_plv'] = np.mean(plv_values)
        features['std_plv'] = np.std(plv_values)

        return features

# Dataset with precomputed features
class EEGFeatureDataset(Dataset):
    def __init__(self, features, labels, texts):
        self.features = features
        self.labels = labels
        self.texts = texts

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]

        # Get text from dictionary if texts is a dictionary
        if isinstance(self.texts, dict):
            text = self.texts.get(label, f"Unknown-{label}")
        else:
            text = self.texts[idx] if idx < len(self.texts) else f"Unknown-{label}"

        # Convert to tensors if not already
        if not isinstance(feature, torch.Tensor):
            feature = torch.tensor(feature, dtype=torch.float32)
        if not isinstance(label, torch.Tensor) and not isinstance(label, int):
            label = torch.tensor(label, dtype=torch.long)

        return feature, label, text

# STEP 3: Classification Model
class EEGClassifier(nn.Module):
    def __init__(self, input_dim, n_classes, hidden_dims=[4096, 2048, 1024],
                 seq_length=None, n_channels=None, dropout_rate=0.5):
        super(EEGClassifier, self).__init__()

        self.input_dim = input_dim
        self.n_classes = n_classes
        self.seq_length = seq_length
        self.n_channels = n_channels

        # Option to reshape as temporal sequence if seq_length and n_channels are provided
        self.reshape_input = seq_length is not None and n_channels is not None

        # Input normalization layer
        self.input_norm = nn.BatchNorm1d(input_dim)

        # PART 1: CNN FEATURE EXTRACTION (if seq_length and n_channels provided)
        if self.reshape_input:
            # CNN for spatial-temporal feature extraction
            self.conv_block = nn.Sequential(
                nn.Conv2d(1, 32, kernel_size=(1, 16), stride=(1, 2), padding=(0, 7)),
                nn.BatchNorm2d(32),
                nn.ELU(),
                nn.Conv2d(32, 64, kernel_size=(n_channels, 1), stride=1, padding=0),
                nn.BatchNorm2d(64),
                nn.ELU(),
                nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4)),
                nn.Dropout(dropout_rate)
            )

            # Calculate output size after convolutions
            conv_output_size = self._calculate_conv_output_size()
            lstm_input_size = conv_output_size

            # LSTM for temporal dynamics
            self.lstm = nn.LSTM(
                input_size=64,  # Number of features per timestep (output channels from CNN)
                hidden_size=128,
                num_layers=2,
                batch_first=True,
                dropout=dropout_rate,
                bidirectional=True
            )

            # Self-attention mechanism for temporal focus
            self.attention = SelfAttention(256)  # 256 = 128*2 (bidirectional)

            # Set the input dimension for dense layers
            dense_input_dim = 256
        else:
            # If no reshape, use attention on flat input
            self.attention = nn.Sequential(
                nn.Linear(input_dim, input_dim // 4),
                nn.LeakyReLU(0.2),
                nn.Linear(input_dim // 4, input_dim),
                nn.Sigmoid()
            )
            dense_input_dim = input_dim

        # PART 2: DENSE NETWORK PATHWAY
        layers = []
        prev_dim = dense_input_dim

        for i, hidden_dim in enumerate(hidden_dims):
            # Dense block with residual connection if dimensions match
            if prev_dim == hidden_dim:
                layers.append(ResidualBlock(prev_dim, hidden_dim, dropout_rate))
            else:
                layers.append(DenseBlock(prev_dim, hidden_dim, dropout_rate))

            prev_dim = hidden_dim

            # Add Squeeze-and-Excitation blocks for feature recalibration
            if i < len(hidden_dims) - 1:  # Not for the last layer
                layers.append(SEBlock(hidden_dim))

        self.feature_layers = nn.Sequential(*layers)

        # Multi-head output with ensemble averaging
        self.heads = nn.ModuleList([
            nn.Linear(prev_dim, n_classes) for _ in range(3)
        ])

        # Initialize weights
        self.apply(self._init_weights)

    def _calculate_conv_output_size(self):
        # Calculate output size after convolutions
        # This is a placeholder - actual calculation depends on your exact architecture
        length_after_conv = ((self.seq_length - 16 + 2*7) // 2) + 1
        length_after_pool = length_after_conv // 4
        return 64 * length_after_pool  # 64 channels

    def _init_weights(self, m):
        if isinstance(m, (nn.Linear, nn.Conv2d)):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Apply input normalization
        if not self.reshape_input:
            x = self.input_norm(x)

            # Apply attention mechanism
            attn = self.attention(x)
            x = x * attn

            # Pass through feature layers
            features = self.feature_layers(x)
        else:
            # Reshape input to [batch, 1, channels, time]
            batch_size = x.size(0)
            x = x.view(batch_size, 1, self.n_channels, self.seq_length)

            # Pass through CNN
            x = self.conv_block(x)  # -> [batch, 64, 1, reduced_time]

            # Reshape for LSTM: [batch, time, features]
            x = x.squeeze(2).permute(0, 2, 1)  # -> [batch, reduced_time, 64]

            # Pass through LSTM
            x, _ = self.lstm(x)  # -> [batch, reduced_time, 256]

            # Apply self-attention
            x, _ = self.attention(x)  # -> [batch, 256]

            # Pass through feature layers
            features = self.feature_layers(x)

        # Ensemble predictions from multiple heads
        logits = torch.stack([head(features) for head in self.heads])
        logits = torch.mean(logits, dim=0)

        return logits

    def predict_proba(self, x):
        logits = self.forward(x)
        return torch.softmax(logits, dim=1)


# Helper blocks for enhanced architecture

class ResidualBlock(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_rate=0.4):
        super(ResidualBlock, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.norm = nn.BatchNorm1d(out_dim)
        self.activation = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        residual = x
        out = self.linear(x)
        out = self.norm(out)
        out = self.activation(out)
        out = self.dropout(out)
        out += residual
        return out


class DenseBlock(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_rate=0.4):
        super(DenseBlock, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.norm = nn.BatchNorm1d(out_dim)
        self.activation = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = self.linear(x)
        out = self.norm(out)
        out = self.activation(out)
        out = self.dropout(out)
        return out


class SEBlock(nn.Module):
    """Squeeze-and-Excitation block for feature recalibration"""
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c = x.size()
        y = self.avg_pool(x.unsqueeze(-1)).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.squeeze(-1)


class SelfAttention(nn.Module):
    """Self-attention mechanism for sequential data"""
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.scale = hidden_dim ** 0.5

    def forward(self, x):
        # x shape: [batch, seq_len, hidden_dim]
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # Calculate attention scores
        attn_scores = torch.bmm(q, k.transpose(1, 2)) / self.scale
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Apply attention to values
        context = torch.bmm(attn_probs, v)

        # Global feature vector (attention-weighted sum)
        global_feat = torch.sum(context, dim=1)

        return global_feat, attn_probs

# STEP 4: Data Preprocessing Helper - OPTIMIZED FOR MEMORY
# STEP 4: Data Preprocessing Helper - OPTIMIZED FOR MEMORY
def prepare_dataset_with_features(dataset, batch_size=64, device='cuda'):
    """Pre-compute features for the dataset in memory-efficient batches"""
    feature_extractor = FeatureExtractor()

    processed_features = []
    labels = []
    label_texts = {}

    # Process in batches to reduce memory usage
    num_batches = (len(dataset) + batch_size - 1) // batch_size

    for batch_idx in tqdm(range(num_batches), desc="Extracting features in batches"):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(dataset))

        batch_features = []
        batch_labels = []

        for idx in range(start_idx, end_idx):
            eeg_data, label_idx, label_text = dataset[idx]

            # Convert to numpy if needed
            if isinstance(eeg_data, torch.Tensor):
                eeg_data_np = eeg_data.cpu().numpy()
            else:
                eeg_data_np = eeg_data

            # Extract features
            features = feature_extractor.extract_features(eeg_data_np)

            batch_features.append(torch.tensor(features, dtype=torch.float32))
            batch_labels.append(label_idx)

            # Store label text mapping
            label_texts[label_idx] = label_text

        processed_features.extend(batch_features)
        labels.extend(batch_labels)

        # Force garbage collection after each batch
        gc.collect()

        # Clear CUDA cache if using GPU
        if device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Get label texts from dataset if available
    if not label_texts and hasattr(dataset, 'get_label_texts'):
        label_texts = dataset.get_label_texts()

    return processed_features, labels, label_texts

# STEP 5: Training Function - MEMORY OPTIMIZED
def train_model(model, train_loader, val_loader, num_epochs=500, learning_rate=0.001,
               weight_decay=1e-5, device="cpu", class_weights=None):

    model.to(device)

    # Initialize optimizer with weight decay for regularization
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # Loss function with class weights if provided
    if class_weights is not None:
        class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
        criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        criterion = nn.CrossEntropyLoss()

    # Track metrics
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    # For early stopping
    best_val_loss = float('inf')
    patience = 10
    no_improve_epoch = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for features, labels, _ in progress_bar:
            # Move tensors to device
            features = features.to(device)
            labels = labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            train_loss += loss.item()

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{100 * correct / total:.2f}%"
            })

            # Clear GPU memory after each batch
            del features, labels, outputs, loss, predicted
            if device == 'cuda' and torch.cuda.is_available():
                torch.cuda.empty_cache()

        # Calculate epoch metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100 * correct / total
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")

            for features, labels, _ in progress_bar:
                features = features.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(features)
                loss = criterion(outputs, labels)

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_loss += loss.item()

                # Store predictions for metrics
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # Update progress bar
                progress_bar.set_postfix({
                    'loss': f"{loss.item():.4f}",
                    'acc': f"{100 * correct / total:.2f}%"
                })

                # Clear GPU memory
                del features, labels, outputs, loss, predicted
                if device == 'cuda' and torch.cuda.is_available():
                    torch.cuda.empty_cache()

        # Calculate epoch metrics
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = 100 * correct / total
        val_losses.append(epoch_val_loss)
        val_accs.append(epoch_val_acc)

        # Learning rate scheduler step
        scheduler.step(epoch_val_loss)

        # Print epoch summary
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        print(f"Learning rate: {optimizer.param_groups[0]['lr']:.6f}")

        # Check if this is the best model
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = {k: v.cpu().detach() for k, v in model.state_dict().items()}
            no_improve_epoch = 0
            print("New best model saved!")

            # Print classification report
            print("\nClassification Report:")
            print(classification_report(all_labels, all_preds))
        else:
            no_improve_epoch += 1

        # Early stopping check
        if no_improve_epoch >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

        # Force garbage collection after each epoch
        gc.collect()
        if device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Load best model weights
    model.load_state_dict(best_model_state)
    model.to(device)  # Make sure model is on the correct device

    # Plot training curves
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.close()

    return model

# STEP 6: Evaluation Function - MEMORY OPTIMIZED
def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for features, labels, _ in tqdm(test_loader, desc="Evaluating"):
            features = features.to(device)

            # Get predictions
            outputs = model(features)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Store results (move to CPU to save GPU memory)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

            # Clear GPU memory
            del features, outputs, probs, predicted
            if device == 'cuda' and torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)

    # Generate classification report
    report = classification_report(all_labels, all_preds)

    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()

    return accuracy, report, cm, np.array(all_probs)

def main():
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Check device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Enable memory tracking for PyTorch
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

    # Create or use existing label mapping
    label_mapping_file = 'labels_txt.txt'

    # Set up paths for feature storage
    features_path = {
        'session_1': "features_session_1.pt",
        'session_2': "features_session_2.pt",
        'combined': "features_combined.pt"
    }

    # Track whether we need to recompute the combined features
    need_to_combine = False

    # Process both sessions sequentially
    session_files = [
        "/content/drive/MyDrive/session_1.pth",
        "/content/drive/MyDrive/session_2.pth"
    ]

    # Process each session separately
    for i, session_file in enumerate(session_files):
        session_name = f"session_{i+1}"
        session_feature_path = features_path[session_name]

        # Check if features for this session already exist
        if os.path.exists(session_feature_path):
            print(f"Loading pre-computed features from {session_feature_path}...")
            features_data = torch.load(session_feature_path)
            if 'features' not in features_data or 'labels' not in features_data or 'texts' not in features_data:
                print(f"Invalid feature file format for {session_feature_path}. Will recompute.")
                need_to_combine = True
                os.remove(session_feature_path)
            else:
                print(f"Loaded features with shape {len(features_data['features'])}")
        else:
            # If feature file doesn't exist, we need to process and later combine
            need_to_combine = True

            # Load data for this session
            print(f"Loading data from {session_file}...")
            try:
                # Load in CPU memory to avoid GPU memory usage during loading
                session_data = torch.load(session_file, map_location='cpu', weights_only=False)
                print(f"Loaded {len(session_data['dataset'])} EEG samples with {len(set(session_data['labels']))} unique classes")

                # Create EEG dataset with preprocessing
                preprocessor = EEGPreprocessor(sampling_rate=256)  # Adjust sampling rate to match your data
                dataset = EEGDataset(session_data, label_mapping_file, transform=preprocessor)

                # Free up memory from raw data
                del session_data
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()
                    print(f"GPU memory after data loading: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

                # Extract features from the dataset in batches
                print(f"Preprocessing and extracting features for {session_name} in batches...")
                features, labels, texts = prepare_dataset_with_features(dataset, batch_size=32, device=device)

                # Save features to disk
                print(f"Saving extracted features to {session_feature_path}...")
                torch.save({
                    'features': features,
                    'labels': labels,
                    'texts': texts
                }, session_feature_path)
                print("Features saved successfully!")

                # Free up memory
                del dataset, features, labels, texts
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()
                    print(f"GPU memory after feature extraction: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

            except Exception as e:
                print(f"Error processing {session_file}: {e}")
                continue

    # Check if we need to recombine features
    if need_to_combine or not os.path.exists(features_path['combined']):
        print("Combining features from all sessions...")
        all_features = []
        all_labels = []
        all_texts = {}

        # Load and combine features from each session
        for i in range(len(session_files)):
            session_name = f"session_{i+1}"
            session_feature_path = features_path[session_name]

            if os.path.exists(session_feature_path):
                print(f"Loading features from {session_feature_path} for combining...")
                features_data = torch.load(session_feature_path)

                # Add features and labels
                all_features.extend(features_data['features'])
                all_labels.extend(features_data['labels'])

                # Merge text dictionaries
                if isinstance(features_data['texts'], dict):
                    all_texts.update(features_data['texts'])
                else:
                    # Handle case where texts are in list format
                    for j, label in enumerate(features_data['labels']):
                        if j < len(features_data['texts']):
                            all_texts[label] = features_data['texts'][j]

                # Clear memory
                del features_data
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()

        # Save combined features
        print(f"Total combined features: {len(all_features)}")
        print(f"Saving combined features to {features_path['combined']}...")
        torch.save({
            'features': all_features,
            'labels': all_labels,
            'texts': all_texts
        }, features_path['combined'])
        print("Combined features saved successfully!")
    else:
        # Load combined features
        print(f"Loading pre-computed combined features from {features_path['combined']}...")
        combined_data = torch.load(features_path['combined'])
        all_features = combined_data['features']
        all_labels = combined_data['labels']
        all_texts = combined_data['texts']
        print(f"Loaded combined features with shape {len(all_features)}")

        # Clear memory
        del combined_data
        gc.collect()

        if device.type == 'cuda':
            torch.cuda.empty_cache()

    # Create feature dataset from combined data
    feature_dataset = EEGFeatureDataset(all_features, all_labels, all_texts)

    # Free up memory that's no longer needed
    del all_features, all_labels
    gc.collect()

    if device.type == 'cuda':
        torch.cuda.empty_cache()
        print(f"GPU memory before training: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

    # Split data with stratification
    train_idx, temp_idx = train_test_split(
        range(len(feature_dataset)),
        test_size=0.3,
        random_state=42,
        stratify=feature_dataset.labels
    )

    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,  # 50% of temp_idx, resulting in 15% of original data
        random_state=42,
        stratify=[feature_dataset.labels[i] for i in temp_idx]
    )

    # Create samplers
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # Create data loaders with appropriate batch size
    # Smaller batch size can help with memory usage
    batch_size = 16 if device.type == 'cuda' else 32
    train_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=train_sampler, pin_memory=(device.type=='cuda'))
    val_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=val_sampler, pin_memory=(device.type=='cuda'))
    test_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=test_sampler, pin_memory=(device.type=='cuda'))

    # Get feature dimension from the first sample
    feature_dim = feature_dataset[0][0].shape[0]
    n_classes = len(set(feature_dataset.labels))

    print(f"Feature dimension: {feature_dim}")
    print(f"Number of classes: {n_classes}")

    # Calculate class weights for imbalanced data
    class_counts = np.bincount(feature_dataset.labels)
    class_weights = 1.0 / class_counts
    class_weights = class_weights / np.sum(class_weights) * len(class_counts)
    print(f"Class weights: {class_weights}")

    # Create model
    model = EEGClassifier(
        input_dim=feature_dim,
        n_classes=n_classes
        # hidden_dims=[512, 256, 128]  # Adjust architecture as needed
    )

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total model parameters: {total_params:,}")

    # Train model
    print("\nTraining model...")
    trained_model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=500,  # Adjust as needed
        learning_rate=0.0005,
        weight_decay=5e-5,
        device=device,
        class_weights=class_weights
    )

    # Evaluate model
    print("\nEvaluating model on test set...")
    accuracy, report, cm, probabilities = evaluate_model(trained_model, test_loader, device)

    print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")
    # print("\nClassification Report:")
    # print(report)

    # Save model
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'feature_dim': feature_dim,
        'n_classes': n_classes,
        'accuracy': accuracy
    }, "eeg_classifier_model.pt")

    print("\nModel saved successfully!")

    print("\nEEG classification pipeline complete!")

if __name__ == "__main__":
    main()

Using device: cuda
Initial GPU memory: 16.25 MB
Loading pre-computed features from features_session_1.pt...
Loaded features with shape 31950
Loading pre-computed features from features_session_2.pt...
Loaded features with shape 31900
Loading pre-computed combined features from features_combined.pt...
Loaded combined features with shape 63850
GPU memory before training: 16.25 MB
Feature dimension: 2114
Number of classes: 80
Class weights: [0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 1.06400665 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Total model parameters: 24,272,902

Training model...


Epoch 1/500 [Val]: 100%|██████████| 599/599 [00:02<00:00, 269.38it/s, loss=5.6759, acc=2.20%]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie


Epoch 1/500
Train Loss: 5.3126, Train Acc: 1.59%
Val Loss: 4.7420, Val Acc: 2.20%
Learning rate: 0.000500
New best model saved!

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       120
           1       0.00      0.00      0.00       120
           2       0.00      0.00      0.00       120
           3       0.00      0.00      0.00       120
           4       1.00      0.01      0.02       120
           5       0.00      0.00      0.00       120
           6       0.00      0.00      0.00       120
           7       0.03      0.01      0.01       120
           8       0.02      0.10      0.04       120
           9       0.00      0.00      0.00       120
          10       0.09      0.01      0.02       120
          11       0.00      0.00      0.00       120
          12       0.00      0.00      0.00       120
          13       0.06      0.02      0.03       120
          14       0.00      0.00   

...


Epoch 48/500
Train Loss: 1.8308, Train Acc: 48.97%
Val Loss: 2.6922, Val Acc: 30.20%
Learning rate: 0.000125


Epoch 49/500 [Val]: 100%|██████████| 599/599 [00:02<00:00, 248.91it/s, loss=3.1359, acc=30.02%]



Epoch 49/500
Train Loss: 1.8006, Train Acc: 49.89%
Val Loss: 2.7162, Val Acc: 30.02%
Learning rate: 0.000125
Early stopping triggered after 49 epochs

Evaluating model on test set...


Evaluating: 100%|██████████| 599/599 [00:01<00:00, 545.95it/s]



Final Test Accuracy: 29.19%

Model saved successfully!

EEG classification pipeline complete!


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import scipy.signal as signal
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import os
import pandas as pd
from scipy.stats import skew, kurtosis
import seaborn as sns
import gc

# STEP 1: Data Loading and Preprocessing
class EEGDataset(Dataset):
    def __init__(self, data_dict, label_mapping_file=None, transform=None):
        self.dataset = data_dict['dataset']
        self.labels_list = data_dict['labels']  # List of label IDs
        self.transform = transform

        # Load the mapping from label ID to text if provided
        self.label_id_to_text = {}
        if label_mapping_file and os.path.exists(label_mapping_file):
            with open(label_mapping_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        label_id = parts[0]
                        label_text = ' '.join(parts[1:])
                        self.label_id_to_text[label_id] = label_text

        # Create mapping from label ID to index
        unique = sorted(set(self.labels_list))
        self.label_to_idx = {lbl:i for i,lbl in enumerate(unique)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        eeg_data = sample['eeg_data']  # Shape: [channels, time_points]
        label_id = sample['label']  # This is a string ID like 'n02510455'

        # Get label text if available
        label_text = self.label_id_to_text.get(label_id, label_id)

        # Convert string label to numerical index
        label_idx = self.label_to_idx.get(label_id, 0)

        if self.transform:
            eeg_data = self.transform(eeg_data)

        return eeg_data, label_idx, label_text

# EEG Signal Preprocessing
class EEGPreprocessor:
    def __init__(self, sampling_rate=1000, notch_freq=50, bandpass_low=0.5, bandpass_high=70):
        self.sampling_rate = sampling_rate
        self.notch_freq = notch_freq
        self.bandpass_low = bandpass_low
        self.bandpass_high = bandpass_high

        # Pre-compute filter coefficients to avoid recomputation
        self.sos = signal.butter(
            N=4,
            Wn=[self.bandpass_low, self.bandpass_high],
            btype='bandpass',
            fs=self.sampling_rate,
            output='sos'
        )
        self.b_notch, self.a_notch = signal.iirnotch(self.notch_freq, 30, self.sampling_rate)

    def __call__(self, eeg_data):
        # Convert to numpy if it's a tensor
        if isinstance(eeg_data, torch.Tensor):
            eeg_data = eeg_data.numpy()

        # Transpose to [time, channels] for easier processing
        eeg_data = eeg_data.T

        # Apply bandpass filter
        eeg_filtered = self._bandpass_filter(eeg_data)

        # Apply notch filter (to remove power line interference)
        eeg_filtered = self._notch_filter(eeg_filtered)

        # Re-reference to common average
        eeg_filtered = self._common_average_reference(eeg_filtered)

        # Z-score normalization
        eeg_normalized = self._normalize(eeg_filtered)

        # Transpose back to [channels, time]
        return torch.tensor(eeg_normalized.T, dtype=torch.float32)

    def _bandpass_filter(self, data):
        # Apply forward-backward filtering for zero phase distortion
        return signal.sosfiltfilt(self.sos, data, axis=0)

    def _notch_filter(self, data):
        return signal.filtfilt(self.b_notch, self.a_notch, data, axis=0)

    def _common_average_reference(self, data):
        # Subtract the mean across all channels at each time point
        return data - np.mean(data, axis=1, keepdims=True)

    def _normalize(self, data):
        # Z-score normalization for each channel
        return (data - np.mean(data, axis=0)) / (np.std(data, axis=0) + 1e-10)

# STEP 2: Feature Extraction
class FeatureExtractor:
    def __init__(self, sampling_rate=1000):
        self.sampling_rate = sampling_rate

        # Define frequency bands
        self.freq_bands = {
          'delta': (1, 4),       # Adjusted lower bound to reduce DC components
          'theta': (4, 8),       # Standard theta for cognitive processing
          'alpha_low': (8, 10),  # Lower alpha - attention/inhibition
          'alpha_high': (10, 13),# Higher alpha - semantic processing
          'beta_low': (13, 20),  # Lower beta - motor preparation
          'beta_high': (20, 30), # Higher beta - active processing/cognition
          'gamma_low': (30, 60), # Expanded gamma_low for visual processing
          'gamma_mid': (60, 90), # Added gamma_mid for binding
          'gamma_high': (90, 120)# Higher gamma for fine perceptual binding
      }

    def extract_features(self, eeg_data):
        """
        Extract time and frequency domain features from EEG data

        Args:
            eeg_data: EEG data of shape [channels, time_points]

        Returns:
            features: Dictionary of extracted features
        """
        features = {}

        # Time domain features
        features.update(self._extract_time_domain_features(eeg_data))

        # Frequency domain features
        features.update(self._extract_frequency_domain_features(eeg_data))

        # Connectivity features - can improve classification accuracy
        features.update(self._extract_connectivity_features(eeg_data))

        # Convert dictionary to vector
        feature_vector = []
        for key, value in features.items():
            if isinstance(value, np.ndarray):
                feature_vector.append(value.flatten())
            else:
                feature_vector.append(np.array([value]).flatten())

        return np.concatenate(feature_vector)

    def _extract_time_domain_features(self, eeg_data):
        features = {}

        # Statistical features
        features['mean'] = np.mean(eeg_data, axis=1)
        features['var'] = np.var(eeg_data, axis=1)
        features['skewness'] = skew(eeg_data, axis=1)
        features['kurtosis'] = kurtosis(eeg_data, axis=1)
        features['max'] = np.max(eeg_data, axis=1)
        features['min'] = np.min(eeg_data, axis=1)
        features['peak_to_peak'] = features['max'] - features['min']  # Reuse computed values
        features['rms'] = np.sqrt(np.mean(np.square(eeg_data), axis=1))
        features['zero_crossings'] = np.sum(np.diff(np.signbit(eeg_data), axis=1), axis=1)

        # Hjorth parameters
        features.update(self._compute_hjorth_parameters(eeg_data))

        return features

    def _compute_hjorth_parameters(self, eeg_data):
        """Compute Hjorth parameters: Activity, Mobility, and Complexity"""
        features = {}

        # First derivative
        diff1 = np.diff(eeg_data, axis=1)
        # Second derivative
        diff2 = np.diff(diff1, axis=1)

        # Activity: variance of the signal
        features['activity'] = np.var(eeg_data, axis=1)

        # Mobility: sqrt(variance of first derivative / variance of signal)
        var_diff1 = np.var(diff1, axis=1)
        mobility1 = np.sqrt(var_diff1 / (features['activity'] + 1e-10))
        features['mobility'] = mobility1

        # Complexity: mobility of first derivative / mobility of signal
        var_diff2 = np.var(diff2, axis=1)
        mobility2 = np.sqrt(var_diff2 / (var_diff1 + 1e-10))
        features['complexity'] = mobility2 / (mobility1 + 1e-10)

        return features

    def _extract_frequency_domain_features(self, eeg_data):
        features = {}

        # Compute power spectral density with Welch's method
        nperseg = min(256, eeg_data.shape[1] // 4)  # Adaptive window size
        freqs, psd = signal.welch(eeg_data, fs=self.sampling_rate,
                                 nperseg=nperseg,
                                 noverlap=nperseg // 2,
                                 axis=1)

        # Calculate total power once
        total_power = np.sum(psd, axis=1) + 1e-10

        # Band powers and their ratios
        for band_name, (low_freq, high_freq) in self.freq_bands.items():
            # Find frequencies in the band
            idx_band = np.logical_and(freqs >= low_freq, freqs <= high_freq)
            # Calculate band power
            band_power = np.sum(psd[:, idx_band], axis=1)
            features[f'{band_name}_power'] = band_power

            # Calculate relative band power
            features[f'{band_name}_rel_power'] = band_power / total_power

        # Spectral edge frequency (95%)
        features['sef_95'] = self._compute_spectral_edge_frequency(freqs, psd, 0.95)

        # Spectral entropy
        features['spectral_entropy'] = self._compute_spectral_entropy(psd)

        # Spectral peak frequency and power
        peak_freqs = freqs[np.argmax(psd, axis=1)]
        peak_powers = np.max(psd, axis=1)
        features['peak_freq'] = peak_freqs
        features['peak_power'] = peak_powers

        return features

    def _compute_spectral_edge_frequency(self, freqs, psd, edge=0.95):
        """Compute frequency below which edge% of power resides"""
        sef = np.zeros(psd.shape[0])
        for i in range(psd.shape[0]):
            # Cumulative sum of PSD
            cumsum = np.cumsum(psd[i]) / (np.sum(psd[i]) + 1e-10)
            # Find frequency below which edge% of power resides
            idx = np.where(cumsum >= edge)[0]
            if len(idx) > 0:
                sef[i] = freqs[idx[0]]
            else:
                sef[i] = freqs[-1]
        return sef

    def _compute_spectral_entropy(self, psd):
        """Compute spectral entropy"""
        entropy = np.zeros(psd.shape[0])
        for i in range(psd.shape[0]):
            # Normalize PSD
            psd_norm = psd[i] / (np.sum(psd[i]) + 1e-10)
            # Calculate entropy
            entropy[i] = -np.sum(psd_norm * np.log2(psd_norm + 1e-10))
        return entropy

    def _extract_connectivity_features(self, eeg_data):
        """Extract connectivity features between EEG channels"""
        features = {}

        # Number of channels
        n_channels = eeg_data.shape[0]

        # Calculate correlation matrix
        corr_matrix = np.corrcoef(eeg_data)

        # Extract upper triangle (excluding diagonal)
        upper_tri_idx = np.triu_indices(n_channels, k=1)
        correlations = corr_matrix[upper_tri_idx]

        # Basic statistics of correlations
        features['mean_corr'] = np.mean(correlations)
        features['std_corr'] = np.std(correlations)
        features['max_corr'] = np.max(correlations)
        features['min_corr'] = np.min(correlations)

        # Phase synchronization - simplified version using Hilbert transform
        analytic_signal = signal.hilbert(eeg_data, axis=1)
        instantaneous_phase = np.angle(analytic_signal)

        # Calculate phase differences between adjacent channels
        phase_diff = np.zeros((n_channels-1,) + instantaneous_phase.shape[1:])
        for i in range(n_channels-1):
            phase_diff[i] = instantaneous_phase[i+1] - instantaneous_phase[i]

        # Phase locking value (PLV)
        plv_values = np.abs(np.mean(np.exp(1j * phase_diff), axis=1))
        features['mean_plv'] = np.mean(plv_values)
        features['std_plv'] = np.std(plv_values)

        return features

# Dataset with precomputed features
class EEGFeatureDataset(Dataset):
    def __init__(self, features, labels, texts):
        self.features = features
        self.labels = labels
        self.texts = texts

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]

        # Get text from dictionary if texts is a dictionary
        if isinstance(self.texts, dict):
            text = self.texts.get(label, f"Unknown-{label}")
        else:
            text = self.texts[idx] if idx < len(self.texts) else f"Unknown-{label}"

        # Convert to tensors if not already
        if not isinstance(feature, torch.Tensor):
            feature = torch.tensor(feature, dtype=torch.float32)
        if not isinstance(label, torch.Tensor) and not isinstance(label, int):
            label = torch.tensor(label, dtype=torch.long)

        return feature, label, text

# STEP 3: Classification Model
class EEGClassifier(nn.Module):
    def __init__(self, input_dim, n_classes, hidden_dims=[4096, 2048, 1024],
                 seq_length=None, n_channels=None, dropout_rate=0.3):
        super(EEGClassifier, self).__init__()

        self.input_dim = input_dim
        self.n_classes = n_classes
        self.seq_length = seq_length
        self.n_channels = n_channels

        # Option to reshape as temporal sequence if seq_length and n_channels are provided
        self.reshape_input = seq_length is not None and n_channels is not None

        # Input normalization layer
        self.input_norm = nn.BatchNorm1d(input_dim)

        # PART 1: CNN FEATURE EXTRACTION (if seq_length and n_channels provided)
        if self.reshape_input:
            # CNN for spatial-temporal feature extraction
            self.conv_block = nn.Sequential(
                nn.Conv2d(1, 32, kernel_size=(1, 16), stride=(1, 2), padding=(0, 7)),
                nn.BatchNorm2d(32),
                nn.ELU(),
                nn.Conv2d(32, 64, kernel_size=(n_channels, 1), stride=1, padding=0),
                nn.BatchNorm2d(64),
                nn.ELU(),
                nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4)),
                nn.Dropout(dropout_rate)
            )

            # Calculate output size after convolutions
            conv_output_size = self._calculate_conv_output_size()
            lstm_input_size = conv_output_size

            # LSTM for temporal dynamics
            self.lstm = nn.LSTM(
                input_size=64,  # Number of features per timestep (output channels from CNN)
                hidden_size=128,
                num_layers=2,
                batch_first=True,
                dropout=dropout_rate,
                bidirectional=True
            )

            # Self-attention mechanism for temporal focus
            self.attention = SelfAttention(256)  # 256 = 128*2 (bidirectional)

            # Set the input dimension for dense layers
            dense_input_dim = 256
        else:
            # If no reshape, use attention on flat input
            self.attention = nn.Sequential(
                nn.Linear(input_dim, input_dim // 4),
                nn.LeakyReLU(0.2),
                nn.Linear(input_dim // 4, input_dim),
                nn.Sigmoid()
            )
            dense_input_dim = input_dim

        # PART 2: DENSE NETWORK PATHWAY
        layers = []
        prev_dim = dense_input_dim

        for i, hidden_dim in enumerate(hidden_dims):
            # Dense block with residual connection if dimensions match
            if prev_dim == hidden_dim:
                layers.append(ResidualBlock(prev_dim, hidden_dim, dropout_rate))
            else:
                layers.append(DenseBlock(prev_dim, hidden_dim, dropout_rate))

            prev_dim = hidden_dim

            # Add Squeeze-and-Excitation blocks for feature recalibration
            if i < len(hidden_dims) - 1:  # Not for the last layer
                layers.append(SEBlock(hidden_dim))

        self.feature_layers = nn.Sequential(*layers)

        # Multi-head output with ensemble averaging
        self.heads = nn.ModuleList([
            nn.Linear(prev_dim, n_classes) for _ in range(3)
        ])

        # Initialize weights
        self.apply(self._init_weights)

    def _calculate_conv_output_size(self):
        # Calculate output size after convolutions
        # This is a placeholder - actual calculation depends on your exact architecture
        length_after_conv = ((self.seq_length - 16 + 2*7) // 2) + 1
        length_after_pool = length_after_conv // 4
        return 64 * length_after_pool  # 64 channels

    def _init_weights(self, m):
        if isinstance(m, (nn.Linear, nn.Conv2d)):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Apply input normalization
        if not self.reshape_input:
            x = self.input_norm(x)

            # Apply attention mechanism
            attn = self.attention(x)
            x = x * attn

            # Pass through feature layers
            features = self.feature_layers(x)
        else:
            # Reshape input to [batch, 1, channels, time]
            batch_size = x.size(0)
            x = x.view(batch_size, 1, self.n_channels, self.seq_length)

            # Pass through CNN
            x = self.conv_block(x)  # -> [batch, 64, 1, reduced_time]

            # Reshape for LSTM: [batch, time, features]
            x = x.squeeze(2).permute(0, 2, 1)  # -> [batch, reduced_time, 64]

            # Pass through LSTM
            x, _ = self.lstm(x)  # -> [batch, reduced_time, 256]

            # Apply self-attention
            x, _ = self.attention(x)  # -> [batch, 256]

            # Pass through feature layers
            features = self.feature_layers(x)

        # Ensemble predictions from multiple heads
        logits = torch.stack([head(features) for head in self.heads])
        logits = torch.mean(logits, dim=0)

        return logits

    def predict_proba(self, x):
        logits = self.forward(x)
        return torch.softmax(logits, dim=1)


# Helper blocks for enhanced architecture

class ResidualBlock(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_rate=0.4):
        super(ResidualBlock, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.norm = nn.BatchNorm1d(out_dim)
        self.activation = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        residual = x
        out = self.linear(x)
        out = self.norm(out)
        out = self.activation(out)
        out = self.dropout(out)
        out += residual
        return out


class DenseBlock(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_rate=0.4):
        super(DenseBlock, self).__init__()
        self.linear = nn.Linear(in_dim, out_dim)
        self.norm = nn.BatchNorm1d(out_dim)
        self.activation = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        out = self.linear(x)
        out = self.norm(out)
        out = self.activation(out)
        out = self.dropout(out)
        return out


class SEBlock(nn.Module):
    """Squeeze-and-Excitation block for feature recalibration"""
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c = x.size()
        y = self.avg_pool(x.unsqueeze(-1)).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.squeeze(-1)


class SelfAttention(nn.Module):
    """Self-attention mechanism for sequential data"""
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.scale = hidden_dim ** 0.5

    def forward(self, x):
        # x shape: [batch, seq_len, hidden_dim]
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        # Calculate attention scores
        attn_scores = torch.bmm(q, k.transpose(1, 2)) / self.scale
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Apply attention to values
        context = torch.bmm(attn_probs, v)

        # Global feature vector (attention-weighted sum)
        global_feat = torch.sum(context, dim=1)

        return global_feat, attn_probs

# STEP 4: Data Preprocessing Helper - OPTIMIZED FOR MEMORY
def prepare_dataset_with_features(dataset, batch_size=64, device='cuda'):
    """Pre-compute features for the dataset in memory-efficient batches"""
    feature_extractor = FeatureExtractor()

    processed_features = []
    labels = []
    label_texts = {}

    # Process in batches to reduce memory usage
    num_batches = (len(dataset) + batch_size - 1) // batch_size

    for batch_idx in tqdm(range(num_batches), desc="Extracting features in batches"):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(dataset))

        batch_features = []
        batch_labels = []

        for idx in range(start_idx, end_idx):
            eeg_data, label_idx, label_text = dataset[idx]

            # Convert to numpy if needed
            if isinstance(eeg_data, torch.Tensor):
                eeg_data_np = eeg_data.cpu().numpy()
            else:
                eeg_data_np = eeg_data

            # Extract features
            features = feature_extractor.extract_features(eeg_data_np)

            batch_features.append(torch.tensor(features, dtype=torch.float32))
            batch_labels.append(label_idx)

            # Store label text mapping
            label_texts[label_idx] = label_text

        processed_features.extend(batch_features)
        labels.extend(batch_labels)

        # Force garbage collection after each batch
        gc.collect()

        # Clear CUDA cache if using GPU
        if device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Get label texts from dataset if available
    if not label_texts and hasattr(dataset, 'get_label_texts'):
        label_texts = dataset.get_label_texts()

    return processed_features, labels, label_texts

# STEP 5: Training Function - MEMORY OPTIMIZED
def train_model(model, train_loader, val_loader, num_epochs=500, learning_rate=0.001,
               weight_decay=1e-5, device="cpu", class_weights=None):

    model.to(device)

    # Initialize optimizer with weight decay for regularization
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # Loss function with class weights if provided
    if class_weights is not None:
        class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
        criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        criterion = nn.CrossEntropyLoss()

    # Track metrics
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    # For early stopping
    best_val_loss = float('inf')
    patience = 10
    no_improve_epoch = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for features, labels, _ in progress_bar:
            # Move tensors to device
            features = features.to(device)
            labels = labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            train_loss += loss.item()

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{100 * correct / total:.2f}%"
            })

            # Clear GPU memory after each batch
            del features, labels, outputs, loss, predicted
            if device == 'cuda' and torch.cuda.is_available():
                torch.cuda.empty_cache()

        # Calculate epoch metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100 * correct / total
        train_losses.append(epoch_train_loss)
        train_accs.append(epoch_train_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            progress_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")

            for features, labels, _ in progress_bar:
                features = features.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(features)
                loss = criterion(outputs, labels)

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_loss += loss.item()

                # Store predictions for metrics
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # Update progress bar
                progress_bar.set_postfix({
                    'loss': f"{loss.item():.4f}",
                    'acc': f"{100 * correct / total:.2f}%"
                })

                # Clear GPU memory
                del features, labels, outputs, loss, predicted
                if device == 'cuda' and torch.cuda.is_available():
                    torch.cuda.empty_cache()

        # Calculate epoch metrics
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = 100 * correct / total
        val_losses.append(epoch_val_loss)
        val_accs.append(epoch_val_acc)

        # Learning rate scheduler step
        scheduler.step(epoch_val_loss)

        # Print epoch summary
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        print(f"Learning rate: {optimizer.param_groups[0]['lr']:.6f}")

        # Check if this is the best model
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = {k: v.cpu().detach() for k, v in model.state_dict().items()}
            no_improve_epoch = 0
            print("New best model saved!")

            # Print classification report
            print("\nClassification Report:")
            print(classification_report(all_labels, all_preds))
        else:
            no_improve_epoch += 1

        # Early stopping check
        if no_improve_epoch >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

        # Force garbage collection after each epoch
        gc.collect()
        if device == 'cuda' and torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Load best model weights
    model.load_state_dict(best_model_state)
    model.to(device)  # Make sure model is on the correct device

    # Plot training curves
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.close()

    return model

# STEP 6: Evaluation Function - MEMORY OPTIMIZED
def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for features, labels, _ in tqdm(test_loader, desc="Evaluating"):
            features = features.to(device)

            # Get predictions
            outputs = model(features)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Store results (move to CPU to save GPU memory)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probs.extend(probs.cpu().numpy())

            # Clear GPU memory
            del features, outputs, probs, predicted
            if device == 'cuda' and torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)

    # Generate classification report
    report = classification_report(all_labels, all_preds)

    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Plot confusion matrix
    # plt.figure(figsize=(10, 8))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    # plt.xlabel('Predicted Label')
    # plt.ylabel('True Label')
    # plt.title('Confusion Matrix')
    # plt.savefig('confusion_matrix.png')
    # plt.close()

    return accuracy, report, cm, np.array(all_probs)

def main():
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Check device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Enable memory tracking for PyTorch
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        print(f"Initial GPU memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

    # Create or use existing label mapping
    label_mapping_file = 'labels_txt.txt'

    # Set up paths for feature storage
    features_path = {
        'session_1': "features_session_1.pt",
        'session_2': "features_session_2.pt",
        'combined': "features_combined.pt"
    }

    # Track whether we need to recompute the combined features
    need_to_combine = False

    # Process both sessions sequentially
    session_files = [
        "/content/drive/MyDrive/session_1.pth",
        "/content/drive/MyDrive/session_2.pth"
    ]

    # Process each session separately
    for i, session_file in enumerate(session_files):
        session_name = f"session_{i+1}"
        session_feature_path = features_path[session_name]

        # Check if features for this session already exist
        if os.path.exists(session_feature_path):
            print(f"Loading pre-computed features from {session_feature_path}...")
            features_data = torch.load(session_feature_path)
            if 'features' not in features_data or 'labels' not in features_data or 'texts' not in features_data:
                print(f"Invalid feature file format for {session_feature_path}. Will recompute.")
                need_to_combine = True
                os.remove(session_feature_path)
            else:
                print(f"Loaded features with shape {len(features_data['features'])}")
        else:
            # If feature file doesn't exist, we need to process and later combine
            need_to_combine = True

            # Load data for this session
            print(f"Loading data from {session_file}...")
            try:
                # Load in CPU memory to avoid GPU memory usage during loading
                session_data = torch.load(session_file, map_location='cpu', weights_only=False)
                print(f"Loaded {len(session_data['dataset'])} EEG samples with {len(set(session_data['labels']))} unique classes")

                # Create EEG dataset with preprocessing
                preprocessor = EEGPreprocessor(sampling_rate=1000)  # Adjust sampling rate to match your data
                dataset = EEGDataset(session_data, label_mapping_file, transform=preprocessor)

                # Free up memory from raw data
                del session_data
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()
                    print(f"GPU memory after data loading: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

                # Extract features from the dataset in batches
                print(f"Preprocessing and extracting features for {session_name} in batches...")
                features, labels, texts = prepare_dataset_with_features(dataset, batch_size=32, device=device)

                # Save features to disk
                print(f"Saving extracted features to {session_feature_path}...")
                torch.save({
                    'features': features,
                    'labels': labels,
                    'texts': texts
                }, session_feature_path)
                print("Features saved successfully!")

                # Free up memory
                del dataset, features, labels, texts
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()
                    print(f"GPU memory after feature extraction: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

            except Exception as e:
                print(f"Error processing {session_file}: {e}")
                continue

    # Check if we need to recombine features
    if need_to_combine or not os.path.exists(features_path['combined']):
        print("Combining features from all sessions...")
        all_features = []
        all_labels = []
        all_texts = {}

        # Load and combine features from each session
        for i in range(len(session_files)):
            session_name = f"session_{i+1}"
            session_feature_path = features_path[session_name]

            if os.path.exists(session_feature_path):
                print(f"Loading features from {session_feature_path} for combining...")
                features_data = torch.load(session_feature_path)

                # Add features and labels
                all_features.extend(features_data['features'])
                all_labels.extend(features_data['labels'])

                # Merge text dictionaries
                if isinstance(features_data['texts'], dict):
                    all_texts.update(features_data['texts'])
                else:
                    # Handle case where texts are in list format
                    for j, label in enumerate(features_data['labels']):
                        if j < len(features_data['texts']):
                            all_texts[label] = features_data['texts'][j]

                # Clear memory
                del features_data
                gc.collect()

                if device.type == 'cuda':
                    torch.cuda.empty_cache()

        # Save combined features
        print(f"Total combined features: {len(all_features)}")
        print(f"Saving combined features to {features_path['combined']}...")
        torch.save({
            'features': all_features,
            'labels': all_labels,
            'texts': all_texts
        }, features_path['combined'])
        print("Combined features saved successfully!")
    else:
        # Load combined features
        print(f"Loading pre-computed combined features from {features_path['combined']}...")
        combined_data = torch.load(features_path['combined'])
        all_features = combined_data['features']
        all_labels = combined_data['labels']
        all_texts = combined_data['texts']
        print(f"Loaded combined features with shape {len(all_features)}")

        # Clear memory
        del combined_data
        gc.collect()

        if device.type == 'cuda':
            torch.cuda.empty_cache()

    # Create feature dataset from combined data
    feature_dataset = EEGFeatureDataset(all_features, all_labels, all_texts)

    # Free up memory that's no longer needed
    del all_features, all_labels
    gc.collect()

    if device.type == 'cuda':
        torch.cuda.empty_cache()
        print(f"GPU memory before training: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

    # Split data with stratification
    train_idx, temp_idx = train_test_split(
        range(len(feature_dataset)),
        test_size=0.3,
        random_state=42,
        stratify=feature_dataset.labels
    )

    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=0.5,  # 50% of temp_idx, resulting in 15% of original data
        random_state=42,
        stratify=[feature_dataset.labels[i] for i in temp_idx]
    )

    # Create samplers
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # Create data loaders with appropriate batch size
    # Smaller batch size can help with memory usage
    batch_size = 16 if device.type == 'cuda' else 32
    train_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=train_sampler, pin_memory=(device.type=='cuda'))
    val_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=val_sampler, pin_memory=(device.type=='cuda'))
    test_loader = DataLoader(feature_dataset, batch_size=batch_size, sampler=test_sampler, pin_memory=(device.type=='cuda'))

    # Get feature dimension from the first sample
    feature_dim = feature_dataset[0][0].shape[0]
    n_classes = len(set(feature_dataset.labels))

    print(f"Feature dimension: {feature_dim}")
    print(f"Number of classes: {n_classes}")

    # Calculate class weights for imbalanced data
    class_counts = np.bincount(feature_dataset.labels)
    class_weights = 1.0 / class_counts
    class_weights = class_weights / np.sum(class_weights) * len(class_counts)
    print(f"Class weights: {class_weights}")

    # Create model
    model = EEGClassifier(
        input_dim=feature_dim,
        n_classes=n_classes
        # hidden_dims=[512, 256, 128]  # Adjust architecture as needed
    )

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total model parameters: {total_params:,}")

    # Train model
    print("\nTraining model...")
    trained_model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=500,  # Adjust as needed
        learning_rate=0.0005,
        weight_decay=1e-5,
        device=device,
        class_weights=class_weights
    )

    # Evaluate model
    print("\nEvaluating model on test set...")
    accuracy, report, cm, probabilities = evaluate_model(trained_model, test_loader, device)

    print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")
    # print("\nClassification Report:")
    # print(report)

    # Save model
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'feature_dim': feature_dim,
        'n_classes': n_classes,
        'accuracy': accuracy
    }, "eeg_classifier_model.pt")

    print("\nModel saved successfully!")

    print("\nEEG classification pipeline complete!")

if __name__ == "__main__":
    main()

Using device: cuda
Initial GPU memory: 16.25 MB
Loading pre-computed features from features_session_1.pt...
Loaded features with shape 31950
Loading pre-computed features from features_session_2.pt...
Loaded features with shape 31900
Loading pre-computed combined features from features_combined.pt...
Loaded combined features with shape 63850
GPU memory before training: 16.25 MB
Feature dimension: 2114
Number of classes: 80
Class weights: [0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 1.06400665 0.99750623 0.99750623 0.99750623 0.99750623
 0.99750623 0.99750623

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Total model parameters: 24,272,902

Training model...


Epoch 1/500 [Val]: 100%|██████████| 599/599 [00:02<00:00, 297.47it/s, loss=4.4330, acc=3.28%]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie


Epoch 1/500
Train Loss: 5.0208, Train Acc: 2.00%
Val Loss: 4.6019, Val Acc: 3.28%
Learning rate: 0.000500
New best model saved!

Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.02      0.03       120
           1       0.20      0.01      0.02       120
           2       0.25      0.01      0.02       120
           3       0.00      0.00      0.00       120
           4       0.16      0.03      0.04       120
           5       0.00      0.00      0.00       120
           6       0.00      0.00      0.00       120
           7       0.09      0.02      0.03       120
           8       0.05      0.02      0.02       120
           9       0.17      0.02      0.03       120
          10       0.00      0.00      0.00       120
          11       0.03      0.08      0.04       120
          12       0.04      0.03      0.04       120
          13       0.01      0.03      0.02       120
          14       0.00      0.00   

...


Epoch 33/500
Train Loss: 0.7018, Train Acc: 78.62%
Val Loss: 2.5464, Val Acc: 39.44%
Learning rate: 0.000125


Epoch 34/500 [Val]: 100%|██████████| 599/599 [00:02<00:00, 256.69it/s, loss=3.9435, acc=39.24%]



Epoch 34/500
Train Loss: 0.6625, Train Acc: 79.83%
Val Loss: 2.6649, Val Acc: 39.24%
Learning rate: 0.000125
Early stopping triggered after 34 epochs

Evaluating model on test set...


Evaluating: 100%|██████████| 599/599 [00:01<00:00, 411.14it/s]



Final Test Accuracy: 38.19%

Model saved successfully!

EEG classification pipeline complete!
